In [1]:
import pandas as pd

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

# Concating training and testing data together to perform operation on both of them together
df_full = pd.concat([df_train, df_test], ignore_index=True)

### Checking the data

In [2]:
df_full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [4]:
df_full.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [5]:
df_full.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

### Performing Operations on Dataset

#### ---> Notice that Cabin column have lot of rows with null value, thus we can safely drop it
#### ---> Also Name and Ticket column could be excluded too
#### ---> Lets combine the number of sibling and parents column together and then drop the prior

In [6]:
df_full['Sib&Par'] = (df_full['SibSp'] + df_full['Parch'])
df_full.columns  # Checking if new column has been added successfully

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Sib&Par'],
      dtype='object')

In [7]:
df_full.drop(['SibSp', 'Parch', 'Ticket', 'Cabin', 'Name'], axis=1, inplace=True)
df_full.columns   # Checking if the columns have been dropped successfully

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked',
       'Sib&Par'],
      dtype='object')

In [8]:
# Dividing the processed dataset into training and testing data

df_train = df_full[:891]
df_test = df_full[891:].reset_index(drop=True)
df_test.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Sib&Par
0,892,NaN,3,male,34.5,7.8292,Q,0
1,893,NaN,3,female,47.0,7.0000,S,1
2,894,NaN,2,male,62.0,9.6875,Q,0
3,895,NaN,3,male,27.0,8.6625,S,0
4,896,NaN,3,female,22.0,12.2875,S,2


### Making Pipeline 

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

number_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

category_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])
number_attributes=['Age', 'Fare', 'Sib&Par']
category_attributes=['Sex', 'Embarked', 'Pclass']

preprocess_Pipeline = ColumnTransformer([
    ('number', number_pipeline, number_attributes),
    ('category', category_pipeline, category_attributes),
])

In [10]:
# Creating the final training data

X_train = preprocess_Pipeline.fit_transform(
        df_train[number_attributes+category_attributes]
)
X_train

array([[-0.56573646, -0.50244517,  0.05915988, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.66386103,  0.78684529,  0.05915988, ...,  1.        ,
         0.        ,  0.        ],
       [-0.25833709, -0.48885426, -0.56097483, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.1046374 , -0.17626324,  1.29942929, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25833709, -0.04438104, -0.56097483, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.20276197, -0.49237783, -0.56097483, ...,  0.        ,
         0.        ,  1.        ]])

In [11]:
Y_train = df_train['Survived']    # Creating Label

In [12]:
# Creating the final testing dataset

X_test = preprocess_Pipeline.fit_transform(
        df_test[number_attributes+category_attributes]
)
X_test

array([[ 0.38623105, -0.49741333, -0.5534426 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.37137004, -0.51227801,  0.10564289, ...,  0.        ,
         0.        ,  1.        ],
       [ 2.55353683, -0.46410047, -0.5534426 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 0.70147553, -0.50779638, -0.5534426 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.20485235, -0.49345515, -0.5534426 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.20485235, -0.23695704,  0.76472838, ...,  0.        ,
         0.        ,  1.        ]])

### Using SVC for prediction

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, Y_train, cv=10)
svm_scores.mean()  

0.8226841448189763

In [14]:
svm_clf.fit(X_train, Y_train)
Y_test_pred = svm_clf.predict(X_test)

# We need dtype=int32 for kaggle submission
Y_test_pred = Y_test_pred.astype('int32')

# Changing Y_test_pred from nparray to DataFrame
Y_test_pred = pd.DataFrame(Y_test_pred, columns=['Survived'])

In [15]:
# Making a new DataFrame with intended columns

kaggle_df = pd.DataFrame()
kaggle_df = pd.concat([kaggle_df, df_test.loc[:,'PassengerId'], Y_test_pred], axis=1)
kaggle_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [16]:
kaggle_df.to_csv('Kaggle_Titanic_Prediction.csv', index=False)

### Thank You!!